In [2]:
%matplotlib inline
from __future__ import division
import numpy as np
from fatiando import utils
from fatiando.seismic.wavefd import Ricker, ElasticPSV, ElasticSH
from fatiando.vis import mpl
import fatiando

In [3]:
fatiando.__version__

'0.3-193-ga08a1fe'

In [4]:
shape = (600, 600)
dens = np.ones(shape)*2200
vp = np.ones(shape)*4000
vs = np.ones(shape)*3000

In [ ]:
sim = ElasticPSV